In [15]:
import jieba
import jieba.analyse
import pandas as pd
import numpy as np
import re
from collections import Counter

In [16]:
orgdata = pd.read_csv('t187ap03_L.csv')
orgdata.head()

,出表日期,公司代號,公司名稱,公司簡稱,外國企業註冊地國,產業別,住址,營利事業統一編號,董事長,總經理,...,過戶電話,過戶地址,簽證會計師事務所,簽證會計師1,簽證會計師2,英文簡稱,英文通訊地址,傳真機號碼,電子郵件信箱,網址
0,1111119,1101,臺灣水泥股份有限公司,台泥,－,1,台北市中山北路2段113號,11913502,張安平,張安平,...,66365566,台北市重慶南路一段83號5樓,勤業眾信聯合會計師事務所,黃惠敏,郭政弘,TCC,"No.113, Sec.2, Zhongshan N. Rd.,Taipei City 10...",(02)2531-6529,finance@taiwancement.com,http://www.taiwancement.com
1,1111119,1102,亞洲水泥股份有限公司,亞泥,－,1,台北市大安區敦化南路2段207號30、31樓,3244509,徐旭東,李坤炎,...,02-7753-1699,新北市板橋區新站路16號13樓,勤業眾信聯合會計師事務所,戴信維,陳培德,ACC,"30-31F., No.207, Sec. 2, Dunhua S. Rd., Da' an...",02-2378-5191,service@acc.com.tw,www.acc.com.tw
2,1111119,1103,嘉新水泥股份有限公司,嘉泥,－,1,台北市中山北路2段96號,11892801,張剛綸,王立心,...,(02)2702-3999,台北市敦化南路2段97號地下2樓,勤業眾信聯合會計師事務所,陳薔旬,梁盛泰,CHC,"No.96, Sec. 2, Zhongshan N. Rd., Zhongshan Dis...",(02)2511-4002,ir@chcgroup.com.tw,www.chcgroup.com.tw
3,1111119,1104,環球水泥股份有限公司,環泥,－,1,台北市南京東路二段125號10樓,7568009,侯博義,侯智升,...,02-23816288,台北市博愛路17號3樓,勤業眾信聯合會計師事務所,楊朝欽,李季珍,UCC,"10th F1., 125 Nanking E.Rd Sec. 2,Taipei Taiwan",02-25075870,ucc@ucctw.com,www.ucctw.com
4,1111119,1108,幸福水泥股份有限公司,幸福,－,1,台北市松江路237號15樓,40601248,陳兩傳,陳兩傳,...,2381-6288,台北市中正區博愛路17號3樓,勤業眾信聯合會計師事務所,陳招美,陳薔旬,LUCKY CEMENT,"15TH FL.,NO.237,SUNG KIANG RD.,TAIPEI,TAIWAN,R...",25048672,"wendy@luckygrp,com.tw",www.luckygrp.com.tw


In [7]:
documents = ['台北市中山北路2段113號	']
# 精確模式
for sentence in documents:
    seg_list = jieba.cut(sentence)
    print('/'.join(seg_list))

台北市/中山北路/2/段/113/號/	


In [19]:
orgdata['seg'] = ""
orgdata.head()

,出表日期,公司代號,公司名稱,公司簡稱,外國企業註冊地國,產業別,住址,營利事業統一編號,董事長,總經理,...,過戶地址,簽證會計師事務所,簽證會計師1,簽證會計師2,英文簡稱,英文通訊地址,傳真機號碼,電子郵件信箱,網址,seg
0,1111119,1101,臺灣水泥股份有限公司,台泥,－,1,台北市中山北路2段113號,11913502,張安平,張安平,...,台北市重慶南路一段83號5樓,勤業眾信聯合會計師事務所,黃惠敏,郭政弘,TCC,"No.113, Sec.2, Zhongshan N. Rd.,Taipei City 10...",(02)2531-6529,finance@taiwancement.com,http://www.taiwancement.com,
1,1111119,1102,亞洲水泥股份有限公司,亞泥,－,1,台北市大安區敦化南路2段207號30、31樓,3244509,徐旭東,李坤炎,...,新北市板橋區新站路16號13樓,勤業眾信聯合會計師事務所,戴信維,陳培德,ACC,"30-31F., No.207, Sec. 2, Dunhua S. Rd., Da' an...",02-2378-5191,service@acc.com.tw,www.acc.com.tw,
2,1111119,1103,嘉新水泥股份有限公司,嘉泥,－,1,台北市中山北路2段96號,11892801,張剛綸,王立心,...,台北市敦化南路2段97號地下2樓,勤業眾信聯合會計師事務所,陳薔旬,梁盛泰,CHC,"No.96, Sec. 2, Zhongshan N. Rd., Zhongshan Dis...",(02)2511-4002,ir@chcgroup.com.tw,www.chcgroup.com.tw,
3,1111119,1104,環球水泥股份有限公司,環泥,－,1,台北市南京東路二段125號10樓,7568009,侯博義,侯智升,...,台北市博愛路17號3樓,勤業眾信聯合會計師事務所,楊朝欽,李季珍,UCC,"10th F1., 125 Nanking E.Rd Sec. 2,Taipei Taiwan",02-25075870,ucc@ucctw.com,www.ucctw.com,
4,1111119,1108,幸福水泥股份有限公司,幸福,－,1,台北市松江路237號15樓,40601248,陳兩傳,陳兩傳,...,台北市中正區博愛路17號3樓,勤業眾信聯合會計師事務所,陳招美,陳薔旬,LUCKY CEMENT,"15TH FL.,NO.237,SUNG KIANG RD.,TAIPEI,TAIWAN,R...",25048672,"wendy@luckygrp,com.tw",www.luckygrp.com.tw,


In [22]:
jieba.analyse.set_stop_words('stopwords.txt')
index = 0
for documents in orgdata['住址']: 
    seg_list = jieba.analyse.extract_tags(documents, topK=150) 
    orgdata['seg'].iloc[index] = seg_list
    index += 1

C:\Users\user\AppData\Local\Temp\ipykernel_5224\1308960687.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orgdata['seg'].iloc[index] = seg_list


In [23]:
orgdata['seg']

0                    [中山北路, 113, 台北市]
1        [敦化南路, 207, 30, 31, 大安, 台北市]
2                     [中山北路, 96, 台北市]
3          [東路, 125, 10, 二段, 台北市, 南京]
4                  [237, 15, 台北市, 松江]
                    ...              
975                [桃園, 八德, 區聯華街, 41]
976         [八德, 八號, 二段, 台北市, 十一, 之一]
977             [八德, 30, 12, 台北市, 松山]
978    [桃園, 市觀, 音區, 大潭, 一鄰, 環科路, 323]
979       [桃園, 市觀, 音區, 1119, 中山路, 一段]
Name: seg, Length: 980, dtype: object

In [26]:
orgdata['city'] = ""
index = 0
for documents in orgdata['住址']:
    date_split = documents.split('市')
    orgdata['city'].iloc[index] = date_split[0]
    index += 1

C:\Users\user\AppData\Local\Temp\ipykernel_5224\1597983714.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orgdata['city'].iloc[index] = date_split[0]


In [31]:
orgdata

,出表日期,公司代號,公司名稱,公司簡稱,外國企業註冊地國,產業別,住址,營利事業統一編號,董事長,總經理,...,簽證會計師事務所,簽證會計師1,簽證會計師2,英文簡稱,英文通訊地址,傳真機號碼,電子郵件信箱,網址,seg,city
0,1111119,1101,臺灣水泥股份有限公司,台泥,－,1,台北市中山北路2段113號,11913502,張安平,張安平,...,勤業眾信聯合會計師事務所,黃惠敏,郭政弘,TCC,"No.113, Sec.2, Zhongshan N. Rd.,Taipei City 10...",(02)2531-6529,finance@taiwancement.com,http://www.taiwancement.com,"[中山北路, 113, 台北市]",台北
1,1111119,1102,亞洲水泥股份有限公司,亞泥,－,1,台北市大安區敦化南路2段207號30、31樓,3244509,徐旭東,李坤炎,...,勤業眾信聯合會計師事務所,戴信維,陳培德,ACC,"30-31F., No.207, Sec. 2, Dunhua S. Rd., Da' an...",02-2378-5191,service@acc.com.tw,www.acc.com.tw,"[敦化南路, 207, 30, 31, 大安, 台北市]",台北
2,1111119,1103,嘉新水泥股份有限公司,嘉泥,－,1,台北市中山北路2段96號,11892801,張剛綸,王立心,...,勤業眾信聯合會計師事務所,陳薔旬,梁盛泰,CHC,"No.96, Sec. 2, Zhongshan N. Rd., Zhongshan Dis...",(02)2511-4002,ir@chcgroup.com.tw,www.chcgroup.com.tw,"[中山北路, 96, 台北市]",台北
3,1111119,1104,環球水泥股份有限公司,環泥,－,1,台北市南京東路二段125號10樓,7568009,侯博義,侯智升,...,勤業眾信聯合會計師事務所,楊朝欽,李季珍,UCC,"10th F1., 125 Nanking E.Rd Sec. 2,Taipei Taiwan",02-25075870,ucc@ucctw.com,www.ucctw.com,"[東路, 125, 10, 二段, 台北市, 南京]",台北
4,1111119,1108,幸福水泥股份有限公司,幸福,－,1,台北市松江路237號15樓,40601248,陳兩傳,陳兩傳,...,勤業眾信聯合會計師事務所,陳招美,陳薔旬,LUCKY CEMENT,"15TH FL.,NO.237,SUNG KIANG RD.,TAIPEI,TAIWAN,R...",25048672,"wendy@luckygrp,com.tw",www.luckygrp.com.tw,"[237, 15, 台北市, 松江]",台北
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,1111119,9944,新麗企業股份有限公司,新麗,－,20,桃園市八德區聯華街41號,33375909,簡瑞瑞,簡榮富,...,國富浩華聯合會計師事務所,邵朝彬,黃千真,Shinih,"No.41, Lianhua StBade Dist., Taoyuan City 334,...",(03)3653635,ir@shinih.com,www.shinih.com.tw,"[桃園, 八德, 區聯華街, 41]",桃園
976,1111119,9945,潤泰創新國際股份有限公司,潤泰新,－,20,台北市八德路二段三０八號十一樓之一,12139612,簡滄圳,李志宏,...,資誠聯合會計師事務所,張淑瓊,黃金連,RUENTEX DEVELOP,"11/F,No.308,Section 2, Bade Road,Taipei,Taiwan...",81617968,cmc@mail.ruentex.com.tw,www.rt-develop.com.tw,"[八德, 八號, 二段, 台北市, 十一, 之一]",台北
977,1111119,9946,三發地產股份有限公司(原:金革),三發地產,－,14,台北市松山區八德路3段30號12樓,84466749,鍾鼎晟,鍾鼎晟,...,安侯建業聯合會計師事務所,韓沂璉,黃欣婷,SANFAR,"12F, No.30, Section 3, Bade Road, Songshan Dis...",2570-6985,contact01@sanfar.com.tw,https://www.sanfar.com.tw,"[八德, 30, 12, 台北市, 松山]",台北
978,1111119,9955,佳龍科技工程股份有限公司,佳龍,－,20,桃園市觀音區大潭里一鄰環科路323號,97211972,吳界欣,吳界欣,...,安永聯合會計師事務所,鄭清標,洪茂益,SDTI,"No.323,Huanke Rd.,Guanyin Dist.,Taoyuan 328,Ta...",(03)473-5259,service@sdti.com.tw,www.sdti.com.tw,"[桃園, 市觀, 音區, 大潭, 一鄰, 環科路, 323]",桃園
